In [2]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data/minute_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [3]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
rowID,1587257,793628.0,458201.7724491035,0,1587256
air_pressure,1587257,916.8301266904981,3.0515931266814733,905.0,929.5
air_temp,1587257,61.85144042835004,11.833623786835787,31.64,99.5
avg_wind_direction,1586824,161.96537927331576,95.20811970204107,0.0,359.0
avg_wind_speed,1586824,2.7742720679798536,2.0607577935630346,0.0,32.3
max_wind_direction,1586824,163.40304784903682,92.36723428064296,0.0,359.0
max_wind_speed,1586824,3.399813400856993,2.423167433617141,0.1,36.0
min_wind_direction,1586824,166.82637078844283,97.46274620077628,0.0,359.0
min_wind_speed,1586824,2.1331304542918517,1.7453450849326675,0.0,32.0


In [4]:
df.describe(['air_pressure']).show()

+-------+------------------+
|summary|      air_pressure|
+-------+------------------+
|  count|           1587257|
|   mean| 916.8301266904981|
| stddev|3.0515931266814733|
|    min|             905.0|
|    max|             929.5|
+-------+------------------+



In [5]:
filteredDF = df.filter((df.rowID % 22) == 0)

In [6]:
filteredDF.count()

72149

In [7]:
filteredDF.describe(['air_pressure']).show()

+-------+------------------+
|summary|      air_pressure|
+-------+------------------+
|  count|             72149|
|   mean|  916.830209704934|
| stddev|3.0517146739454795|
|    min|             905.1|
|    max|             929.5|
+-------+------------------+



In [8]:
from pyspark.sql.functions import avg

In [9]:
imputeDF = filteredDF

In [10]:
for x in imputeDF.columns:
    meanValue = filteredDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    imputeDF = imputeDF.na.fill(meanValue, [x])

rowID 793628.0
hpwren_timestamp 1362125288.5993984
air_pressure 916.830209704934
air_temp 61.851424136162734
avg_wind_direction 161.91263414769418
avg_wind_speed 2.772582568425723
max_wind_direction 163.34542857935165
max_wind_speed 3.398079642827498
min_wind_direction 166.44175147666454
min_wind_speed 2.1316852000776767
rain_accumulation 0.00027388146587575883
rain_duration 0.38459832566391305
relative_humidity 47.60542211257255


In [11]:
filteredDF.describe(['air_pressure']).show()

+-------+------------------+
|summary|      air_pressure|
+-------+------------------+
|  count|             72149|
|   mean|  916.830209704934|
| stddev|3.0517146739454795|
|    min|             905.1|
|    max|             929.5|
+-------+------------------+



In [12]:
imputeDF.describe(['air_pressure']).show()

+-------+------------------+
|summary|      air_pressure|
+-------+------------------+
|  count|             72149|
|   mean|  916.830209704934|
| stddev|3.0517146739454795|
|    min|             905.1|
|    max|             929.5|
+-------+------------------+



In [13]:
imputeDF.filter((imputeDF['air_pressure'] > 915) & (imputeDF['air_pressure'] < 928)).count()

52257